!pip install sklearn

# Part-of-speech tagging

### Import libraries

In [1]:
import os
import pandas as pd
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import pickle
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from collections import OrderedDict
from keras.preprocessing.text import Tokenizer




### Define constant variables

In [2]:
train_path = './Dataset/train/'# Should we use: os.path.join(os.getcwd(), "OriginalDataset")
val_path = './Dataset/val/'
test_path = './Dataset/test/'
dataset_folder = os.path.join(os.getcwd(), "OriginalDataset")
original_path = dataset_folder +'/dependency_treebank/'

embedding_dimension = 50


### Download dataste

In [3]:
import urllib.request  #  download files
import zipfile  #  unzip files

def download_dataset():
    dataset_folder = os.path.join(os.getcwd(), "OriginalDataset")

    if not os.path.exists(dataset_folder):
        os.makedirs(dataset_folder)

    url = 'https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip'

    dataset_path = os.path.join(dataset_folder, "dependency_treebank.zip")

    if not os.path.exists(dataset_path):
        urllib.request.urlretrieve(url, dataset_path)
        print("Successful download")

    with zipfile.ZipFile(dataset_path,"r") as zip_ref:
        zip_ref.extractall(dataset_folder)
    print("Successful extraction")

In [4]:
download_dataset()

Successful extraction


### Split dataset

In [5]:
def split_dataset(train_path, val_path, test_path, original_path):
    if not os.path.exists(train_path):
        print("making directory")
        os.makedirs(train_path)
    if not os.path.exists(val_path):
        os.makedirs(val_path)
    if not os.path.exists(test_path):
        os.makedirs(test_path)

    original_dataset = os.listdir(original_path)
    
    original_dataset.sort()
    org_train = original_dataset[0:100]
    org_val = original_dataset[100:150]
    org_test = original_dataset[150:]

    for f in org_train:
        os.rename(original_path+f, train_path+f)
    for f in org_val:
        os.rename(original_path+f, val_path+f)
    for f in org_test:
        os.rename(original_path+f, test_path+f)
    print("Successful spilt")
        

In [6]:
split_dataset(train_path, val_path, test_path, original_path)


Successful spilt


### Preprocessing

In [7]:
def lower(lst): # DO WE NEED THIS?
    lowercase_list = [x.lower() for x in lst]
    return lowercase_list

### Create dataframe

In [8]:
def create_dataframe(dataset_path: str):
    dataframe_rows = []
    documents = os.listdir(dataset_path)
    for document in documents:
        path = os.path.join(dataset_path, document)
        with open(path, 'r') as f:
            a = [[x for x in ln.split()] for ln in f]
        #---------------For splitting on sentence-------------------#
        sentence = []
        sentence_tag = []
        for x in a:
            if x!=[]:
                sentence.append(x[0])
                sentence_tag.append(x[1])
            else:
                dataframe_row = {
                    "text": sentence,
                    "POS": sentence_tag
                }
                sentence = []
                sentence_tag = []
                dataframe_rows.append(dataframe_row)
        #----------------------------------------------------------#
        #------------------To split on document--------------------#
        """
        a2 = [x for x in a if x != []] # OBS! Removing all empty lines in file so we can make an array
        arr = np.array(a2)
        text = list(arr[:, 0])
        tagg = list(arr[:, 1])
        #POStuple =  [tagg[x] for x in range(len(text))] #[(text[x], tagg[x]) for x in range(len(text))]
        document_id = int(document[4:8])
        dataframe_row = {
            "document_id": document_id,
            "text": text,
            "POS": tagg
        }
        dataframe_rows.append(dataframe_row)"""
        #----------------------------------------------------------#
    # transform the list of rows in a proper dataframe
    df = pd.DataFrame(dataframe_rows)
    df = df[["text", "POS"]]

    return df

In [9]:
df_train = create_dataframe(train_path)
df_val = create_dataframe(val_path)
df_test = create_dataframe(test_path)

df_train

,text,POS
0,"[In, reference, to, your, Oct., 9, page-one, a...","[IN, NN, TO, PRP$, NNP, CD, NN, NN, ``, NNP, N..."
1,"[In, part, ,, this, may, reflect, the, fact, t...","[IN, NN, ,, DT, MD, VB, DT, NN, IN, ``, PRP, V..."
2,"[Among, professionals, ,, 76, %, have, a, favo...","[IN, NNS, ,, CD, NN, VBP, DT, JJ, NN, IN, PRP$..."
3,"[While, a, quarter, of, black, voters, disappr...","[IN, DT, NN, IN, JJ, NNS, VBP, IN, NNP, NNP, P..."
4,"[The, statistics, imply, that, three-quarters,...","[DT, NNS, VBP, IN, NNS, IN, NNS, VBP, IN, NNP,..."
...,...,...
1858,"[He, said, authors, of, Scoring, High, ``, scr...","[PRP, VBD, NNS, IN, NNP, NNP, ``, RB, VBP, '',..."
1859,"[When, Scoring, High, first, came, out, in, 19...","[WRB, NNP, NNP, RB, VBD, RB, IN, CD, ,, PRP, V..."
1860,"[McGraw-Hill, was, outraged, .]","[NNP, VBD, JJ, .]"
1861,"[In, a, 1985, advisory, to, educators, ,, McGr...","[IN, DT, CD, NN, TO, NNS, ,, NNP, VBD, NNP, NN..."


### Embed the words using GloVe embedding

embeddings_dict = {}

#### Load glove model

In [10]:
import gensim
import gensim.downloader as gloader

def load_embedding_model(embedding_dimension: int = 50) -> gensim.models.keyedvectors.KeyedVectors:
    download_path = "glove-wiki-gigaword-{}".format(embedding_dimension)
    # Check download
    try:
        emb_model = gloader.load(download_path)
    except ValueError as e:
        print("Invalid embedding model name! Check the embedding dimension:")
        print("Word2Vec: 300")
        print("Glove: 50, 100, 200, 300")
        raise e

    return emb_model

In [11]:
embedding_model = load_embedding_model(embedding_dimension)

### Make tokenizer

In [12]:
word_tokenizer_train = Tokenizer()

# UNCOMMENT IF WE WANT TO INCLUDE GOLVE vocabualry
word_tokenizer_train.fit_on_texts(embedding_model.index_to_key) 
word_tokenizer_train.fit_on_texts(df_train["text"])

x_encoded_train = word_tokenizer_train.texts_to_sequences(df_train["text"])

In [13]:
tag_tokenizer_train = Tokenizer()

tag_tokenizer_train.fit_on_texts(df_train["POS"])
y_encoded_train = tag_tokenizer_train.texts_to_sequences(df_train["POS"])

In [14]:
def pad_idx_seqs(idx_seqs, max_seq_len):
    # Keras provides a convenient padding function; 
    padded_idxs = pad_sequences(sequences=idx_seqs, maxlen=max_seq_len)
    return padded_idxs

In [15]:
def find_max_length(seqs):
    max_len = 0
    for seq in seqs:
        if len(seq) > max_len:
            max_len = len(seq)
    return max_len

In [16]:
x_padded_train = pad_idx_seqs(x_encoded_train, find_max_length(x_encoded_train))
y_padded_train = pad_idx_seqs(y_encoded_train, find_max_length(y_encoded_train))

# NEW METHODE TO GET WORD TO INDX

In [17]:
VOCABULARY_SIZE = len(word_tokenizer_train.word_index) + 1

word2idx_train = word_tokenizer_train.word_index
idx2word_train = word_tokenizer_train.index_word
print(len(word2idx_train))


340107


In [18]:
tag2idx_train = tag_tokenizer_train.word_index
idx2tag_train = tag_tokenizer_train.index_word
len(tag2idx_train)

45

In [19]:
TAG_VOCABULARY_SIZE = len(tag2idx_train) + 1

### Create embedding matrix

In [20]:
# OBS Computes the OOV with random embeddings
def create_embedding_matrix(embedding_model, embedding_dimension, word_to_idx):
    embedding_matrix = np.zeros((len(word_to_idx)+1, embedding_dimension), dtype=np.float32)
    for word, idx in word_to_idx.items():
        try:
            embedding_vector = embedding_model[word]
        except (KeyError, TypeError):
            embedding_vector = np.random.uniform(low=-0.05, high=0.05, size=embedding_dimension)

        embedding_matrix[idx] = embedding_vector
                                
    return embedding_matrix

def expand_embedding_matrix(embedding_model, embedding_dimension, word_to_idx, old_embedding_matrix):
    embedding_matrix = np.zeros((len(word_to_idx), embedding_dimension), dtype=np.float32)
    embedding_matrix[0:len(old_embedding_matrix)] = old_embedding_matrix
    
    for word, idx in word_to_idx.items():
        if idx >= len(old_embedding_matrix):
            try:
                embedding_vector = embedding_model[word]
            except (KeyError, TypeError):
                embedding_vector = np.random.uniform(low=-0.05, high=0.05, size=embedding_dimension)
                
            embedding_matrix[idx] = embedding_vector
    
    return embedding_matrix                                

In [21]:
embedding_matrix = create_embedding_matrix(embedding_model, embedding_dimension, word2idx_train)

#### Define x train and y train

In [22]:
x_train = x_padded_train
y_train = to_categorical(y_padded_train)

### Create baseline model

##### two layers architecture: a Bidirectional LSTM layer and a Dense/Fully-Connected layer on top


https://www.kaggle.com/tanyadayanand/pos-tagging-using-rnn/notebook

Not sure about the parameters etc.. but at least it works to train the model

In [23]:
def create_baseline_model():
    bidirect_model = keras.Sequential()
    bidirect_model.add(layers.Embedding(input_dim = VOCABULARY_SIZE,
                                 output_dim    = embedding_dimension,
                                 input_length  = find_max_length(x_train),
                                 weights       = [embedding_matrix],
                                 trainable = True
                                ))
    bidirect_model.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True)))
    #bidirect_model.add(layers.TimeDistributed(layers.Dense(len(tag_to_idx), activation='softmax')))
    bidirect_model.add(layers.Dense(TAG_VOCABULARY_SIZE, activation='softmax'))
    
    return bidirect_model

In [24]:
baseline_model = create_baseline_model()
baseline_model.summary()

2021-11-19 15:17:14.867566: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 249, 50)           17005400  
                                                                 
 bidirectional (Bidirectiona  (None, 249, 128)         58880     
 l)                                                              
                                                                 
 dense (Dense)               (None, 249, 46)           5934      
                                                                 
Total params: 17,070,214
Trainable params: 17,070,214
Non-trainable params: 0
_________________________________________________________________


In [25]:
baseline_model.compile(loss="mse",
              optimizer="Adam",
              metrics=['mae', "acc"])
baseline_model.fit(x=x_train, batch_size=25, y=y_train, epochs=10)

Epoch 1/10
75/75 [==============================] - 39s 379ms/step - loss: 0.0079 - mae: 0.0166 - acc: 0.9003
Epoch 2/10
75/75 [==============================] - 25s 338ms/step - loss: 0.0020 - mae: 0.0044 - acc: 0.9152
Epoch 3/10
75/75 [==============================] - 25s 336ms/step - loss: 0.0019 - mae: 0.0042 - acc: 0.9288
Epoch 4/10
75/75 [==============================] - 27s 358ms/step - loss: 0.0018 - mae: 0.0038 - acc: 0.9392
Epoch 5/10
75/75 [==============================] - 26s 350ms/step - loss: 0.0015 - mae: 0.0033 - acc: 0.9480
Epoch 6/10
75/75 [==============================] - 25s 338ms/step - loss: 0.0013 - mae: 0.0029 - acc: 0.9519
Epoch 7/10
75/75 [==============================] - 26s 348ms/step - loss: 0.0012 - mae: 0.0027 - acc: 0.9581
Epoch 8/10
75/75 [==============================] - 26s 341ms/step - loss: 0.0010 - mae: 0.0024 - acc: 0.9641
Epoch 9/10
75/75 [==============================] - 28s 371ms/step - loss: 9.2793e-04 - mae: 0.0022 - acc: 0.9693
Epoch 